This notebook visualizes the correspondence between multiple high accuracy fast sampling streams, on two different platforms, while the phones were stationary.

The data is not filtered or interpolated in any way.

Remember to load the `moving`, `high+1sec` data from `Pull_entries_from_server` if you have not already done so.

Note that this loads a LOT of points, so it takes up a lot of memory.
You might want to close other notebooks before loading this.

In [ ]:
import sys
import logging
from uuid import UUID

import emission.core.get_database as edb
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as pydt
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
import numpy as np

logging.getLogger().setLevel(logging.DEBUG)
logging.debug(sys.path)

iphone_ids = [UUID("079e0f1a-c440-3d7c-b0e7-de160f748e35"), UUID("c76a0487-7e5a-3b17-a449-47be666b36f6"), 
              UUID("c528bcd2-a88b-3e82-be62-ef4f2396967a"), UUID("95e70727-a04e-3e33-b7fe-34ab19194f8b")]
android_ids = [UUID("e471711e-bd14-3dbe-80b6-9c7d92ecc296"), UUID("fd7b4c2e-2c8b-3bfa-94f0-d1e3ecbd5fb7"),
               UUID("86842c35-da28-32ed-a90e-2da6663c5c73"), UUID("3bc0f91f-7660-34a2-b005-5c399598a369")]

In [ ]:
list(edb.get_timeseries_db().find({"metadata.key": "config/sensor_config", "data.battery_status": {"$exists": True}}).sort("metadata.write_ts").limit(5))

In [ ]:
iphone_ts = map(lambda u: esta.TimeSeries.get_time_series(u), iphone_ids)
android_ts = map(lambda u: esta.TimeSeries.get_time_series(u), android_ids)

In [ ]:
ranges_df = pd.read_csv("../results_spring_2016/ranges.csv", error_bad_lines=False)

In [ ]:
ranges_df.head()

In [ ]:
regime_grouped_ranges_df = ranges_df.groupby(['state', 'android_regime', 'ios_regime'])

In [ ]:
invalid_df = pd.read_csv("../results_spring_2016/invalid.csv")

In [ ]:
invalid_df

In [ ]:
def get_points_from_ts_for_range(ts, start_ts, end_ts):
    return ts.get_data_df("background/location", estt.TimeQuery('metadata.write_ts', start_ts, end_ts))

In [ ]:
def get_points_for_range(ts, range_df):
    points_range_df = map(lambda row: get_points_from_ts_for_range(ts, row['start_ts'], row['end_ts']), range_df.to_dict('records'))
    return points_range_df

In [ ]:
def get_points_for_group(ts_list, range_df):
    return map(lambda ts: get_points_for_range(ts, range_df), ts_list)

In [ ]:
iphone_high_fast_stationary_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'stationary' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        phone_range_location_df_list = get_points_for_group(iphone_ts, df)
        # one row for each phone
        # each row contains one dataframe
        concat_phone_location_df_list = map(lambda pl: pd.concat(pl), phone_range_location_df_list)
        iphone_high_fast_stationary_df = iphone_high_fast_stationary_df.append(concat_phone_location_df_list)

In [ ]:
android_high_fast_stationary_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'stationary' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        phone_range_location_df_list = get_points_for_group(android_ts, df)
        # one row for each phone
        # each row contains one dataframe
        concat_phone_location_df_list = map(lambda pl: pd.concat(pl), phone_range_location_df_list)
        android_high_fast_stationary_df = android_high_fast_stationary_df.append(concat_phone_location_df_list)
        android_high_fast_stationary_df = android_high_fast_stationary_df.sample(frac=0.001)

In [ ]:
import emission.analysis.plotting.leaflet_osm.our_plotter as lo
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.ipython_helper as ipy

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(iphone_high_fast_stationary_df)]) + 
                 lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(android_high_fast_stationary_df)])])

In [ ]:
iphone_high_fast_loitering_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'loitering' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        phone_range_location_df_list = get_points_for_group(iphone_ts, df)
        # one row for each phone
        # each row contains one dataframe
        concat_phone_location_df_list = map(lambda pl: pd.concat(pl), phone_range_location_df_list)
        iphone_high_fast_loitering_df = iphone_high_fast_loitering_df.append(concat_phone_location_df_list)

In [ ]:
android_high_fast_loitering_df = pd.DataFrame()
for key, df in regime_grouped_ranges_df:
    if key[0] == 'loitering' and key[1] == 'high+1sec':
        # one row for each phone
        # each row contains len(stationary ranges) dataframes
        phone_range_location_df_list = get_points_for_group(android_ts, df)
        # one row for each phone
        # each row contains one dataframe
        concat_phone_location_df_list = map(lambda pl: pd.concat(pl), phone_range_location_df_list)
        android_high_fast_loitering_df = android_high_fast_loitering_df.append(concat_phone_location_df_list)
        android_high_fast_loitering_df = android_high_fast_loitering_df.sample(frac=0.01)

In [ ]:
ipy.inline_maps([lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(iphone_high_fast_loitering_df)]) + 
                 lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(android_high_fast_loitering_df)])])